In [1]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler

In [12]:
batch_s = 20
valid_s = 0.2

transform = transforms.Compose([
    transforms.Resize((227,227)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

train_data = datasets.CIFAR10('data', train = True, download = True, transform = transform)
test_data = datasets.CIFAR10('data', train = False, download = True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
num_train = len(train_data)
num_train

50000

In [14]:
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_s * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_sampler

In [15]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_s,
                sampler = train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_s,
                sampler = valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_s)

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [16]:
train_data[0][0].shape

torch.Size([3, 227, 227])

In [57]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module) :
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2, groups=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1, groups=2)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1, groups=2)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.drop1 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(256*6*6, 2000)
        self.drop2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(2000, 750)
        self.fc3 = nn.Linear(750, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.drop1(self.pool(F.relu(self.conv5(x))))
        x = self.drop2(F.relu(self.fc1(torch.flatten(x, start_dim = 1))))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net()
model.cuda()

print(model)
model(train_data[0][0].reshape(1,3,227,227).cuda())

Net(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=2000, bias=True)
  (drop2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=2000, out_features=750, bias=True)
  (fc3): Linear(in_features=750, out_features=10, bias=True)
)


tensor([[-0.0324, -0.0114,  0.0289, -0.0265,  0.0292,  0.0297,  0.0335, -0.0147,
          0.0045, -0.0189]], device='cuda:0', grad_fn=<AddmmBackward>)

In [58]:
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [59]:
dataiter = next(iter(train_loader))
dataiter[0], dataiter[1] = dataiter[0].cuda(), dataiter[1].cuda()
pred = model(dataiter[0])
loss = loss_f(pred, dataiter[1]) 
loss.backward()
dataiter[0].shape, pred.shape, dataiter[1].shape, loss

(torch.Size([20, 3, 227, 227]),
 torch.Size([20, 10]),
 torch.Size([20]),
 tensor(2.3034, device='cuda:0', grad_fn=<NllLossBackward>))

In [61]:
epocs = 8
valid_loss_min = np.inf

for epoch in range(epocs):
    train_loss = 0.
    valid_loss = 0.
    
    ## train ##
    model.train()
    for data, target in train_loader :
        data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()          #making grads zero
        preds = model(data)           #predicting class
        loss = loss_f(preds, target)   #calculating loss
        loss.backward()               #back propagation
        optimizer.step()              #adding gradients to params
        train_loss += loss.item() * data.size(0)
    
    ##validate##
    model.eval()
    for data, target in valid_loader :
        data, target = data.cuda(), target.cuda()
        
        preds = model(data)
        valid_loss += loss_f(preds, target).item() * data.size(0)
    
    ##avg loss##
    train_loss /= len(train_loader.dataset)
    valid_loss /= len(valid_loader.dataset)
    
    print(f'Epoch {epoch+1} : TrainLoss = {train_loss}, ValidLoss = {valid_loss}')
    
    if valid_loss < valid_loss_min :
        print(f'Better model! {valid_loss_min} ==> {valid_loss}')
        valid_loss_min = valid_loss
        torch.save(model.state_dict(), 'model_alexcifar.pt')

Epoch 1 : TrainLoss = 1.8410667520523072, ValidLoss = 0.4584487931251526
Better model! inf ==> 0.4584487931251526
Epoch 2 : TrainLoss = 1.60603965382576, ValidLoss = 0.3507590305805206
Better model! 0.4584487931251526 ==> 0.3507590305805206
Epoch 3 : TrainLoss = 1.3032896768569946, ValidLoss = 0.2937166684389114
Better model! 0.3507590305805206 ==> 0.2937166684389114
Epoch 4 : TrainLoss = 1.1578171651363374, ValidLoss = 0.2660969559669495
Better model! 0.2937166684389114 ==> 0.2660969559669495
Epoch 5 : TrainLoss = 1.0543515243053436, ValidLoss = 0.25223487298488617
Better model! 0.2660969559669495 ==> 0.25223487298488617
Epoch 6 : TrainLoss = 0.9540956894397735, ValidLoss = 0.2099670972585678
Better model! 0.25223487298488617 ==> 0.2099670972585678
Epoch 7 : TrainLoss = 0.8586887283086777, ValidLoss = 0.2286564083814621
Epoch 8 : TrainLoss = 0.771607297706604, ValidLoss = 0.18397453236579894
Better model! 0.2099670972585678 ==> 0.18397453236579894
Epoch 9 : TrainLoss = 0.7032606570482

In [ ]:
test_loss = 0.0
class_correct = [0. for i in range(10)]
class_total = [0. for i in range(10)]

model.eval()

for data, target in test